**Script Description:** Merge all the individual datasets to one pre-processed dataset

**File Name:** 01_10_Final_Merging.ipynb

**Date:** 2025

**Created by:** Rob Alamgir

#### Import the relevant packages

In [1]:
import os
import glob
import pandas as pd
import numpy as np

#### Import all the relevant datasets

In [2]:
base_dir = "C:/Data_MSc_Thesis/"

# Get file paths of Remotely Sensed Data
Planet_SWC_Data = pd.read_csv("C:/Data_MSc_Thesis/Planet/Planet_SWC_Extracted.csv") 

S1_SAR_VSM_CSV_Files = glob.glob(os.path.join(base_dir, "S1_SAR_VSM/Final_S1_SAR_VSM_Data", "*VSM.csv"))
S1_Backscatter_CSV_Files = glob.glob(os.path.join(base_dir, "S1_SAR_Backscatter", "*.csv"))
S2_Indices_CSV_Files = glob.glob(os.path.join(base_dir, "S2_Indices", "*.csv"))
L8_9_LST_CSV_Files = glob.glob(os.path.join(base_dir, "L8_L9_LST", "*.csv"))
MODIS_CSV_Files = glob.glob(os.path.join(base_dir, "MODIS_LAI", "*.csv"))

# Get file paths of Hybrid Sensed Data
OWASIS_Data = pd.read_csv("C:/Data_MSc_Thesis/OWASIS/OWASIS_NOBV_extracted_data.csv")
BIS_4D_Data = pd.read_csv("C:/Data_MSc_Thesis/BIS_4D_Selected/NOBV_Point_Data_Extracted_V1.csv") 
BOFEK_Data = pd.read_csv("C:/Data_MSc_Thesis/BOFEK/WP_SAT_BOFEK.csv")                             
#BOFEK_Data = pd.read_csv("C:/Data_MSc_Thesis/BOFEK/BOFEK_NOBV.csv")     
# Get file paths of Ground Sensed Data
EC_Tower_Data = pd.read_csv("C:/Data_MSc_Thesis/Pre_Processed_Data_Final/Pre_Processed_Data_All_Locations_V5.csv") 

In [3]:
# Function to read a file and add a Source column
def read_and_add_name(file):
    base_name = os.path.splitext(os.path.basename(file))[0]
    df = pd.read_csv(file)
    df['Source'] = base_name
    return df

# Load and process S1_SAR_VSM data
S1_SAR_VSM_merged = pd.concat([read_and_add_name(file) for file in S1_SAR_VSM_CSV_Files], ignore_index=True)
S1_SAR_VSM_merged.rename(columns={"system:time_start": "Date", "VSM":"S1_VSM", "Source":"Site_ID"}, inplace=True)
S1_SAR_VSM_merged.drop(columns=["pixel_count"], inplace=True)
S1_SAR_VSM_merged['Date'] = pd.to_datetime(S1_SAR_VSM_merged['Date'], errors='coerce')
S1_SAR_VSM_merged = S1_SAR_VSM_merged[S1_SAR_VSM_merged["Date"] >= "2020-01-01"]
S1_SAR_VSM_merged['Site_ID'] = S1_SAR_VSM_merged['Site_ID'].str.replace(r'^S1_SAR_|_VSM$', '', regex=True)

# Load and process S1_Backscatter data
S1_Backscatter_merged = pd.concat([read_and_add_name(file) for file in S1_Backscatter_CSV_Files], ignore_index=True)
S1_Backscatter_merged.rename(columns={"mean": "S1_Backscatter", "stdDev": "S1_Backscatter_SD", "Datetime": "Date"}, inplace=True)
S1_Backscatter_merged.drop(columns=["count", "Source"], inplace=True)
S1_Backscatter_merged['Date'] = pd.to_datetime(S1_Backscatter_merged['Date'], errors='coerce')
S1_Backscatter_merged['Date'] = S1_Backscatter_merged['Date'].dt.date
S1_Backscatter_merged['Date'] = pd.to_datetime(S1_Backscatter_merged['Date'], errors='coerce')

# Read and merge S2_Indices Data
S2_Indices_merged = pd.concat([read_and_add_name(file) for file in S2_Indices_CSV_Files], ignore_index=True)
S2_Indices_merged = S2_Indices_merged.rename(columns={'NDVI': 'S2_NDVI', 'EVI': 'S2_EVI', 'NDMI': 'S2_NDMI', 'Source':'Site_ID'})
S2_Indices_merged.drop(columns=['MNDWI', 'STR'], errors='ignore', inplace=True)
S2_Indices_merged['Date'] = pd.to_datetime(S2_Indices_merged['Date'], errors='coerce')
S2_Indices_merged.dropna(inplace=True)
S2_Indices_merged = S2_Indices_merged[S2_Indices_merged["Date"] >= "2020-01-01"]
S2_Indices_merged['Site_ID'] = S2_Indices_merged['Site_ID'].str.replace(r'^S2_Indices_|$', '', regex=True)

# Read and merge L8_9_LST Data
L8_9_LST_merged = pd.concat([read_and_add_name(file) for file in L8_9_LST_CSV_Files], ignore_index=True)
L8_9_LST_merged = L8_9_LST_merged.rename(columns={'Mean_Surface_Temperature': 'L8_9_LST', 'Source':'Site_ID'})
L8_9_LST_merged['Date'] = pd.to_datetime(L8_9_LST_merged['Date'], errors='coerce')
L8_9_LST_merged.drop(columns=['system:index', '.geo'], errors='ignore', inplace=True)
L8_9_LST_merged = L8_9_LST_merged.loc[:, ~L8_9_LST_merged.columns.duplicated()]
L8_9_LST_merged = L8_9_LST_merged[L8_9_LST_merged["Date"] >= "2020-01-01"]
L8_9_LST_merged['Site_ID'] = L8_9_LST_merged['Site_ID'].str.replace(r'^L8_9_LST_|$', '', regex=True)

# Read and merge MODIS LAI Data
MODIS_LAI_merged = pd.concat([read_and_add_name(file) for file in MODIS_CSV_Files], ignore_index=True)
MODIS_LAI_merged = MODIS_LAI_merged.rename(columns={'Mean_LAI': 'MODIS_LAI', 'Source':'Site_ID'})
MODIS_LAI_merged['Date'] = pd.to_datetime(MODIS_LAI_merged['Date'], errors='coerce')
MODIS_LAI_merged.drop(columns=['system:index', '.geo', 'Unnamed: 0'], errors='ignore', inplace=True)
MODIS_LAI_merged = MODIS_LAI_merged.loc[:, ~MODIS_LAI_merged.columns.duplicated()]
MODIS_LAI_merged = MODIS_LAI_merged[MODIS_LAI_merged["Date"] >= "2020-01-01"]
MODIS_LAI_merged['Site_ID'] = MODIS_LAI_merged['Site_ID'].str.replace(r'^MODIS_LAI_|$', '', regex=True)

# Pre-process Planet_SWC_Data
Planet_SWC_Data.rename(columns={'timestamp': 'Date','swc':'Planet_SWC'}, inplace=True)
Planet_SWC_Data['Date'] = pd.to_datetime(Planet_SWC_Data['Date']).dt.normalize()
Planet_SWC_Data.drop(columns=['Matched_X', 'Matched_Y', 'swc-qf','Source'], errors='ignore', inplace=True)

# Pre-process OWASIS_Data
OWASIS_Data.rename(columns={'Datetime': 'Date'}, inplace=True)
OWASIS_Data['Date'] = pd.to_datetime(OWASIS_Data['Date'], errors='coerce')

# Pre-process BIS_4D_Data
BIS_4D_Data.drop(columns=['Site_no', 'Location_No', 'Longitude', 'Latitude','EPSG_32631_WGS.84_X_m',
                          'EPSG_32631_WGS.84_Y_m','Reproj_X','Reproj_Y','Elevation_m','geometry'], errors='ignore', inplace=True)
# Pre-process BOFEK_Data
BOFEK_Data.drop(columns=['Site_Name', 'BOFEK_2020_PU_Description','Site_Name','Province','Dominant_iProfile'], errors='ignore', inplace=True)

# Pre-process EC Tower Data
EC_Tower_Data['Date'] = pd.to_datetime(EC_Tower_Data['Date'], errors='coerce')

#### Merge all the individual datasets to one dataframe

In [4]:
# List of dataframes with 'Date' column
time_dependent_dfs = [
    S1_SAR_VSM_merged,
    S1_Backscatter_merged,
    Planet_SWC_Data,
    OWASIS_Data,
    S2_Indices_merged,
    L8_9_LST_merged,
    MODIS_LAI_merged,
    EC_Tower_Data
]

# Merge all time-dependent datasets on 'Date' and 'Site_ID'
merged_df = time_dependent_dfs[0]
for df in time_dependent_dfs[1:]:
    merged_df = merged_df.merge(df, on=['Date', 'Site_ID'], how='outer')

# Merge non-time-dependent datasets based on 'Site_ID'
merged_df = merged_df.merge(BIS_4D_Data, on='Site_ID', how='left')
merged_df = merged_df.merge(BOFEK_Data, on='Site_ID', how='left')

#### Re-order the columns of the merged dataframe

In [5]:
# Define the new order for the first columns
first_columns = ['Date', 'year_month', 'DOY', 'Site_ID']

# Get the remaining columns (excluding the ones already selected)
remaining_columns = [col for col in merged_df.columns if col not in first_columns]

# Reorder the dataframe
merged_df = merged_df[first_columns + remaining_columns]

In [6]:
# Get the list of columns
cols = merged_df.columns.to_list()

# Move 'DOY' after 'Date' and 'Site_ID' after 'DOY'
cols.remove('DOY')
cols.remove('Site_ID')
cols.insert(1, 'DOY')  # Insert 'DOY' at index 1 (after 'Date')
cols.insert(2, 'Site_ID')  # Insert 'Site_ID' at index 2 (after 'DOY')

# Reorder the dataframe
merged_df = merged_df[cols]

#### Further dataframe post-processing

In [7]:
merged_df = merged_df[merged_df["Site_ID"] != "ARM"].reset_index(drop=True)
merged_df.drop(columns=["EPSG_32631_WGS 84_X_m", "EPSG_32631_WGS 84_Y_m", "P_minus_ET.1", "P_minus_PET.1","year_week"], inplace=True)

columns_to_divide = [
    "SWCT_1_005", "SWCT_1_015", "SWCT_1_025", "SWCT_1_035", "SWCT_1_045",
    "SWCT_1_055", "SWCT_1_065", "SWCT_1_075", "SWCT_1_085", "SWCT_1_095",
    "SWCT_1_105", "SWCT_1_115"]

merged_df[columns_to_divide] = merged_df[columns_to_divide] / 100
#print(merged_df[columns_to_divide].describe())

merged_df['DOY'] = merged_df['Date'].dt.dayofyear
merged_df['year_month'] = merged_df['Date'].dt.strftime('%Y-%m')
merged_df['week_number'] = merged_df['Date'].dt.isocalendar().week

In [8]:
merged_df.rename(columns={"Planet_SWC": "Planet_SMC", "SWCT_1_015": "SENTEK_SMC", "S1_VSM": "Sentinel_1_SMC"}, inplace=True)

In [9]:
# Check missing values
merged_df.info(verbose=True)
#for col in merged_df.columns:
#    print(col)
#print(merged_df.head(30))
#print(merged_df.tail(30))
#print(merged_df.describe))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36056 entries, 0 to 36055
Data columns (total 109 columns):
 #    Column                       Dtype         
---   ------                       -----         
 0    Date                         datetime64[ns]
 1    DOY                          int32         
 2    Site_ID                      object        
 3    year_month                   object        
 4    Sentinel_1_SMC               float64       
 5    S1_Backscatter               float64       
 6    S1_Backscatter_SD            float64       
 7    Planet_SMC                   float64       
 8    RZSM_OWASIS                  float64       
 9    GWL_OWASIS                   float64       
 10   AWS_OWASIS                   float64       
 11   S2_NDVI                      float64       
 12   S2_EVI                       float64       
 13   S2_NDMI                      float64       
 14   L8_9_LST                     float64       
 15   MODIS_LAI                    float

#### Export the final dataframe to a CSV file

In [10]:
output_path = "C:/Data_MSc_Thesis/Pre_Processed_Data_Final/Pre_Processed_Data_All_Locations_V6.csv"  
merged_df.to_csv(output_path, index=False)

print(f"DataFrame successfully saved to {output_path}")

DataFrame successfully saved to C:/Data_MSc_Thesis/Pre_Processed_Data_Final/Pre_Processed_Data_All_Locations_V6.csv
